In [69]:
import numpy as np
import scipy

In [57]:
def get_reward(s):
    if(s[0]<0 or s[0]>4):
        return(-1)
    if(s[1]<0 or s[1]>4):
        return(-1)
    return(0)
def display(arr):
    np.set_printoptions(precision=5)
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            print("{0:0.2f}".format(arr[i,j]),end="   ")
        print()
def policy_value():
    gamma = 0.9
    gridworld = np.zeros((5,5))
    mx = [-1,0,0,1]
    my = [0,1,-1,0]
    special_states = {}
    special_rewards = {}
    special_states[(0,1)]=(4,1)
    special_rewards[(0,1)]=10
    special_states[(0,3)]=(2,3) 
    special_rewards[(0,3)]=5 
    changes = True
    count = 0
    while(changes):
        count+=1
        changes= False
        for i in range(5):
            for j in range(5):
                total = 0
                best_value = 0
                best_move = 0
                for d in range(4):
                    state = (i,j)
                    dx = mx[d]
                    dy = my[d]
                    dest = (dx,dy) 
                    new_s = np.add(state,dest) if state not in special_states else special_states[state]

                    r = get_reward(new_s) if state not in special_states else special_rewards[state]
                    if(r!=-1):
                        r += gamma*gridworld[new_s[0],new_s[1]]
                    else:
                        new_s = np.copy(state)
                        
                    if(r>best_value):
                        best_value=r
                        best_move=new_s
                
                if(np.abs(best_value-gridworld[i,j])>=0.001*gridworld[i,j]):
                    changes = True
                gridworld[i,j]=best_value
        if(count%10==0):
            display(gridworld)
            print("count",count)
    return(gridworld)
                
            

In [58]:
policy_value()

21.79   24.29   21.86   19.29   17.36   
19.61   21.86   19.68   17.71   15.94   
17.65   19.68   17.71   15.94   14.35   
15.88   17.71   15.94   14.35   12.91   
14.29   15.94   14.35   12.91   11.62   
count 10


array([[21.95416, 24.40413, 21.96371, 19.40413, 17.46371],
       [19.75875, 21.96371, 19.76734, 17.79061, 16.01155],
       [17.78287, 19.76734, 17.79061, 16.01155, 14.41039],
       [16.00459, 17.79061, 16.01155, 14.41039, 12.96935],
       [14.40413, 16.01155, 14.41039, 12.96935, 11.67242]])

## Question 2

In [45]:
def get_linear_equation():
    gamma = 0.9
    # defining possible moves for an agent not in either of the special positions by (mx[i],my[i])
    mx = [-1,0,0,1]
    my = [0,1,-1,0]
    # defining possible moves for agent in the special states A and B. special_states gives destination on any move, and special_rewards gives reward
    special_states = {}
    special_rewards = {}
    special_states[(0,1)]=(4,1)
    special_rewards[(0,1)]=10
    special_states[(0,3)]=(2,3) 
    special_rewards[(0,3)]=5 
    # we will have to solve a system of linear equations of the for ax=b where a is the coefficient matrix and b is the dependent variable value matrix. as there are 25 states, there are 25 V_pi's to solve for, hence a 25x25 matrix of coefficients
    coeffs_matrix = np.zeros((25,25))
    constants_matrix = np.zeros((25,1))
    # iterating over all the states
    for i in range(5):
        for j in range(5):
            # current state
            state = (i,j)
            # we can represent each of the 25 states' coeffs in a 5x5 grid, and later unravel it to a 1x25 list 
            coeffs = np.zeros((5,5))
            # our equation for a state i is: v_i = SUM(prob_action*SUM(p(s',r|s,a)*[r + gamma*v_s'])).
            # here p(s',r|s,a)=1 for a given s,a as there is only one possible successor state (and corresponding reward) for each state s and action a
            coeffs[state]+=1
            results = np.zeros((1,1))
            for d in range(4):
                dx = mx[d]
                dy = my[d]
                dest = (dx,dy)
                new_s = np.add(state,dest) if state not in special_states else special_states[state]
                r = get_reward(new_s) if state not in special_states else special_rewards[state]
                results[0,0]+=0.25*r
                if(r==-1):
                    new_s=np.copy(state)
                coeffs[new_s[0],new_s[1]]+=-0.25*gamma
            #add each state's coeff and constant matrix to the global one
            coeffs_matrix[i*5+j,:]=coeffs.ravel()
            constants_matrix[i*5+j,:]=results.ravel()
    display(coeffs_matrix)
    return(coeffs_matrix,constants_matrix)


In [46]:
a,b=get_linear_equation()

0.55   -0.23   0.00   0.00   0.00   -0.23   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   
0.00   1.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   -0.90   0.00   0.00   0.00   
0.00   -0.23   0.78   -0.23   0.00   0.00   0.00   -0.23   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   
0.00   0.00   0.00   1.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   -0.90   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   
0.00   0.00   0.00   -0.23   0.55   0.00   0.00   0.00   0.00   -0.23   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   
-0.23   0.00   0.00   0.00   0.00   0.78   -0.23   0.00   0.00   0.00   -0.23   0.00   0.00   0.00   0.00   0.0

In [41]:
print(a.shape,b.shape)

(25, 25) (25, 1)


In [42]:
x = np.linalg.solve(a,b)

In [43]:
display(x.reshape(5,5))

3.31   8.79   4.43   5.32   1.49   
1.52   2.99   2.25   1.91   0.55   
0.05   0.74   0.67   0.36   -0.40   
-0.97   -0.44   -0.35   -0.59   -1.18   
-1.86   -1.35   -1.23   -1.42   -1.98   


In [374]:
def get_nonlinear_equation():
    gamma = 0.9
    # defining possible moves for an agent not in either of the special positions by (mx[i],my[i])
    mx = [-1,0,0,1]
    my = [0,1,-1,0]
    # defining possible moves for agent in the special states A and B. special_states gives destination on any move, and special_rewards gives reward
    special_states = {}
    special_rewards = {}
    special_states[(0,1)]=(4,1)
    special_rewards[(0,1)]=10
    special_states[(0,3)]=(2,3) 
    special_rewards[(0,3)]=5 
    # we will have to solve a system of linear equations of the for ax=b where a is the coefficient matrix and b is the dependent variable value matrix. as there are 25 states, there are 25 V_pi's to solve for, hence a 25x25 matrix of coefficients
    coeffs_matrix = np.zeros((100,25))
    constants_matrix = np.zeros((100,1))
    # iterating over all the states
    for i in range(5):
        for j in range(5):
            # current state
            state = (i,j)
            # we can represent each of the 25 states' coeffs in a 5x5 grid, and later unravel it to a 1x25 list 
            coeffs = np.zeros((5,5))
            # our equation for a state i is: v_i = SUM(prob_action*SUM(p(s',r|s,a)*[r + gamma*v_s'])).
            # here p(s',r|s,a)=1 for a given s,a as there is only one possible successor state (and corresponding reward) for each state s and action a
            coeffs[state]+=1
            results = np.zeros((1,1))
            for d in range(4):
                dx = mx[d]
                dy = my[d]
                dest = (dx,dy)
                new_s = np.add(state,dest) if state not in special_states else special_states[state]
                r = get_reward(new_s) if state not in special_states else special_rewards[state]
                results[0,0]+=0.25*r
                if(r==-1):
                    new_s=np.copy(state)
                coeffs[new_s[0],new_s[1]]+=-0.25*gamma
                #add each state's coeff and constant matrix to the global one
                coeffs_matrix[i*5+d+j,:]=coeffs.ravel()
                constants_matrix[i*5+d+j,:]=results.ravel()
#     display(coeffs_matrix)
    return(coeffs_matrix,constants_matrix)


In [375]:
a,b = get_nonlinear_equation()

In [384]:
print(a.shape,b.shape)
c = np.ones((25,1))
# import scipy.optimize as opt
# print(a.T.shape,b.ravel().shape)
x = scipy.optimize.linprog(c,-a,-b)

(100, 25) (100, 1)


In [386]:

x

     con: array([], dtype=float64)
     fun: 4.8563525351304815
 message: 'Optimization terminated successfully.'
     nit: 7
   slack: array([ 2.50000e-01, -3.52163e-13,  2.50000e-01, -7.72715e-14,
        2.50000e-01,  1.90889e-13,  4.79616e-14,  1.90889e-13,
       -3.44169e-15,  1.91026e-13,  1.94335e-13,  6.95000e-14,
        1.94335e-13, -1.45023e-14,  1.95608e-13,  1.97973e-13,
        7.66817e-14,  1.97973e-13,  1.96301e-14,  2.10194e-13,
        2.26518e-13,  2.34344e-14,  2.26518e-13,  5.48485e-14,
        1.70694e-13,  2.50000e-01,  2.49271e-01,  4.99271e-01,
       -0.00000e+00, -0.00000e+00, -0.00000e+00, -0.00000e+00,
       -0.00000e+00, -0.00000e+00, -0.00000e+00, -0.00000e+00,
       -0.00000e+00, -0.00000e+00, -0.00000e+00, -0.00000e+00,
       -0.00000e+00, -0.00000e+00, -0.00000e+00, -0.00000e+00,
       -0.00000e+00, -0.00000e+00, -0.00000e+00, -0.00000e+00,
       -0.00000e+00, -0.00000e+00, -0.00000e+00, -0.00000e+00,
       -0.00000e+00, -0.00000e+00, -0.00000e+

In [341]:
def valid_state(s):
    if(s[0]>=0 and s[0]<4):
        if(s[1]>=0 and s[1]<4):
            return(True)
    return(False)
def terminal(s):
    if(s[0]==0 and s[1]==0):
        return(True)
    if(s[0]==3 and s[1]==3):
        return(True)
    return(False)
def policy_evaluation(gridworld=None,policy=None):
    gamma = 1
    if(gridworld is None):
        gridworld = np.zeros((4,4))
    mx = [-1,0,0,1]
    my = [0,1,-1,0]
    if(policy is None):
        policy = np.ones((4,4,4))*0.25
    changes = True
    count = 0
    while(changes):
        count+=1
        changes= False
        for i in range(4):
            for j in range(4):
                total = 0
                state = (i,j)
                if(terminal(state)):
                        continue
                for d in range(4):
                    dx = mx[d]
                    dy = my[d]
                    dest = (dx,dy) 
                    new_s = np.add(state,dest) 
                    if(not valid_state(new_s)):
                        new_s=state
                    r=-1
#                     if(terminal(new_s)):
#                         r=0
                    r += gamma*gridworld[new_s[0],new_s[1]]
                    total+=policy[i,j,d]*r
                if(np.abs(total-gridworld[i,j])>0.01):
                    changes = True
                gridworld[i,j]=total
        if(count%10==0):
            display(gridworld)
            print("evalutation count",count)
    return(gridworld)

In [342]:
grid = policy_evaluation()

0.00   -7.83   -11.12   -12.23   
-7.83   -10.42   -11.77   -11.86   
-11.12   -11.77   -11.05   -8.81   
-12.23   -11.86   -8.81   0.00   
evalutation count 10
0.00   -11.43   -16.30   -17.93   
-11.43   -14.84   -16.57   -16.61   
-16.30   -16.57   -15.11   -11.84   
-17.93   -16.61   -11.84   0.00   
evalutation count 20
0.00   -12.93   -18.46   -20.30   
-12.93   -16.68   -18.57   -18.59   
-18.46   -18.57   -16.79   -13.10   
-20.30   -18.59   -13.10   0.00   
evalutation count 30
0.00   -13.55   -19.36   -21.29   
-13.55   -17.45   -19.40   -19.41   
-19.36   -19.40   -17.50   -13.62   
-21.29   -19.41   -13.62   0.00   
evalutation count 40
0.00   -13.81   -19.73   -21.71   
-13.81   -17.77   -19.75   -19.75   
-19.73   -19.75   -17.79   -13.84   
-21.71   -19.75   -13.84   0.00   
evalutation count 50
0.00   -13.92   -19.89   -21.88   
-13.92   -17.90   -19.90   -19.90   
-19.89   -19.90   -17.91   -13.93   
-21.88   -19.90   -13.93   0.00   
evalutation count 60


In [345]:
def print_policy(policy):
    mx = [-1,0,0,1]
    my = [0,1,-1,0]
    
    z = np.zeros((4,4))
    for i in range(4):
        for j in range(4):
            val = np.argmax(policy[i,j,:])
    #         print(val)
#             z[i,j]=val
            print("({0},{1})".format(mx[val],my[val]),end=" ")
        print()
#     plt.contour(z)
#     plt.show()
#     print(z)
def policy_improvement(gridworld):
    stable = False
    policy = []
    mx = [-1,0,0,1]
    my = [0,1,-1,0]
    policy = np.ones((4,4,4))*0.25
#     policy[:,:,0]=
    while(not stable):
        stable=True
        for i in range(4):
            for j in range(4):
                state = (i,j)
                if(terminal(state)):
                    continue
                old_action = np.argmax(policy[i,j,:])
                old_actions = np.where(policy[i,j,:]==policy[i,j,:].max())[0]
#                 print(old_actions,i,j)
                best_value = -np.inf
                best_move = None
                for d in range(4):
                    dest = (mx[d],my[d])
                    new_s = np.add(state,dest)
                    if(not valid_state(new_s)):
                        new_s =  state
                    if(terminal(new_s)):
                        r = policy[i,j,d]*(gridworld[new_s[0],new_s[1]])
                    else:
                        r = policy[i,j,d]*(-1+gridworld[new_s[0],new_s[1]])
                    if(r>best_value):
#                         print(best_value,r,d)
                        best_value = r
                        best_move = d
                policy[i,j,:]=0
                policy[i,j,best_move]=1
#                 stable = True
                if(best_move not in old_actions):
                    stable=False
#                     print(best_move,old_action)
                
                        
        if(not stable):
            print("not stable")
            display(gridworld)
            gridworld=policy_evaluation(gridworld,policy)
            display(gridworld)
            print_policy(policy)
    return(policy)
                

In [346]:
p = policy_improvement(grid)
print_policy(p)

(-1,0) (0,-1) (0,-1) (0,-1) 
(-1,0) (0,-1) (-1,0) (-1,0) 
(-1,0) (-1,0) (-1,0) (1,0) 
(-1,0) (-1,0) (0,1) (-1,0) 


-1 0   0 -1   0 -1   0 -1   
-1 0   0 -1   -1 0   -1 0   
-1 0   -1 0   -1 0   1 0   
-1 0   -1 0   0 1   -1 0   


In [387]:
def value_iteration():
    stable = False
    mx = [-1,0,0,1]
    my = [0,1,-1,0]
    gridworld = np.ones((4,4))
    gridworld[0,0]=0
    gridworld[3,3]=0
    
    while(not stable):
        print(gridworld)
        stable = True
        for i in range(4):
            for j in range(4):
                state = (i,j)
                old = gridworld[i,j]
                if(terminal(state)):
                    continue
                best_value = -np.inf
                for d in range(4):
#                     d = int(policy[i,j])
                    dest = (mx[d],my[d])
                    new_s = np.add(state,dest)
                    
                    if(not valid_state(new_s)):
                        new_s =  state
    #                 if(terminal(new_s)):
    #                     r = (gridworld[new_s[0],new_s[1]])
    #                 else:
                    r = -1+gridworld[new_s[0],new_s[1]]
                    if(r>best_value):
    #                         print(best_value,r,d)
                        best_value = r
                        best_move = d    
            
                gridworld[i,j]=best_value
                if(np.abs(old-best_value)>0.01):
                          stable=False
                policy[i][j]=best_move
#                 if():

    return(gridworld)

In [388]:
value_iteration()

[[0. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 0.]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[ 0. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1.  0.]]
[[ 0. -1. -2. -2.]
 [-1. -2. -2. -2.]
 [-2. -2. -2. -1.]
 [-2. -2. -1.  0.]]
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]


array([[ 0., -1., -2., -3.],
       [-1., -2., -3., -2.],
       [-2., -3., -2., -1.],
       [-3., -2., -1.,  0.]])

In [ ]:
mx = [-1,0,0,1]
my = [0,1,-1,0]    
for i in range(4):
    for j in range(4):
        for d in range(4):
            

In [230]:
from scipy.stats import poisson

prob_poi = np.zeros((3,21))
vals = [2,3,4]
for i in range(3):
    for j in range(21):
        prob_poi[i,j]= poisson.pmf(j,vals[i])
prob_poi>0.01

array([[ True,  True,  True,  True,  True,  True,  True, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False],
       [ True,  True,  True,  True,  True,  True,  True,  True, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True, False, False, False, False, False, False, False, False,
        False, False, False]])

In [310]:
def valid_state(s):
    if(s[0]>=0 and s[0]<21):
        if(s[1]>=0 and s[1]<21):
            return(True)
    return(False)
def make_valid(s):
    for i in range(2):
        if(s[i]<0):
            s[i]=0
        if(s[i]>20):
            s[i]=20
    return(s)
def jacks_policy_evaluation(gridworld=None,probs=None):
    gamma = 0.9
    if(gridworld is None):
        gridworld = np.zeros((21,21))
    if(probs is None):
        probs = np.zeros((21,21)) 
#         probs[:,:,0] = 1
    rewards = np.zeros((21,21))

    changes = True
    count = 0
    while(changes):
        count+=1
        changes= False
        for i in range(21):
            for j in range(21):
                total = 0                
                for req_a in range(9):
                    for req_b in range(9):
                        for ret_a in range(9):
                            for ret_b in range(9):
                                state = [i,j]
                                prob = prob_poi[1,req_a]*prob_poi[1,ret_a]*prob_poi[0,ret_b]*prob_poi[2,req_b]
                                add1=10*min(state[0],req_a)
                                state[0]=max(state[0]-req_a,0)
                                add2=10*min(state[1],req_b)
                                state[1]=max(state[1]-req_b,0)
                                state[0]=min(20,ret_a+state[0])
                                state[1]=min(20,ret_b+state[1])
#                                 for d in range(11):
                                d = int(probs[state[0],state[1]])
#                                     if(probs[state[0],state[1],d]>0):
                                r = -2*d +add1+add2
                                if(d<6):
                                    move = (-d,d)
                                else:
                                    r = -2*(d-5) +add1+add2
                                    move = (d-5,5-d)
                                new_s = np.add(state,move)
            #                     print(new_s)
                                if(not valid_state(new_s)):
                                    new_s = make_valid(new_s)
                                r += gamma*gridworld[new_s[0],new_s[1]]
                                total+=prob*r
            
                if(np.abs(total-gridworld[i,j])>1):
                    changes = True
                gridworld[i,j]=total
        print(count,changes)
        if(count%100==0):
#             display(gridworld)
            print("count",count)
        if(count==1000):
            break
    return(gridworld)

In [318]:
grid =jacks_policy_evaluation()
grid
grid2 = np.copy(grid)

1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 True
12 True
13 True
14 True
15 True
16 True
17 True
18 True
19 True
20 True
21 False


In [321]:
def jacks_policy_improvement(gridworld):
    gamma = 0.9
#     gridworld = np.zeros((21,21))
    probs = np.zeros((21,21))
    stable = False
    count = 0
    while(not stable):
        stable=True
        count+=1
        for i in range(21):
            for j in range(21):
                total = 0
                best_move=0
                best_reward = -np.inf               
                for req_a in range(9):
                    for req_b in range(9):
                        for ret_a in range(9):
                            for ret_b in range(9):
                                state = [i,j]
                                add1=10*min(state[0],req_a)
                                state[0]=max(state[0]-req_a,0)
                                add2=10*min(state[1],req_b)
                                state[1]=max(state[1]-req_b,0)
                                state[0]=min(20,ret_a+state[0])
                                state[1]=min(20,ret_b+state[1])
                                prob = prob_poi[1,req_a]*prob_poi[1,ret_a]*prob_poi[0,ret_b]*prob_poi[2,req_b]
                                for d in range(11):
#                                 d = int(probs[state[0],state[1]])
#                                     if(probs[state[0],state[1],d]>0):
                                    r = -2*d +add1+add2
                                    if(d<6):
                                        move = (-d,d)
                                    else:
                                        r = -2*(d-5) +add1+add2
                                        move = (d-5,5-d)
                                    new_s = np.add(state,move) 
                                    if(not valid_state(new_s)):
                                        new_s = make_valid(new_s)
                                    r += gamma*gridworld[new_s[0],new_s[1]]
                                    total=prob*r
                                    if(total>best_reward):
                                        best_reward=total
                                        best_move=d
                old_action = probs[i,j]
                probs[i,j]=best_move
                if(old_action!=best_move):
                    stable=False
        if(not stable):
            gridworld=jacks_policy_evaluation(gridworld,probs)
#         if(count%100==0):
#     #             display(gridworld)
#             print("count",count)
        print(count,stable)
#         plot_policy(probs)
    return(probs)

In [322]:
policy = jacks_policy_improvement(grid)
policy

1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 False
1 False
1 True
2 True
3 True
4 True
5 False
2 False
1 True
2 True
3 True
4 True
5 False
3 False
1 True
2 True
3 True
4 True
5 True
6 False
4 False


KeyboardInterrupt: 

In [314]:
import matplotlib.pyplot as plt
def plot_policy(policy):
#     z = np.zeros((21,21))
#     for i in range(21):
#         for j in range(21):
#             val = np.argmax(policy[i,j,:])
#     #         print(val)
#             if(val>5):
#                 val = 5-val
#             z[i,j]=val
    plt.contour(policy)
    plt.show()
#     print(z)

In [ ]:
plot_